# Orientação: Caso deseje executar no Google Colab

Faça a importação do notebook e leve também o diretório **data**.

https://colab.research.google.com/

Descrição: Criar um motor de busca em documentos públicos recuperados do site do tribunal.

*  ouvidoria_tjdft.txt
*  sobre_nos.txt
*  canais_comunicacao.txt
*  cartilha_justica_comunitaria.txt
*  carta_servicos.txt
*  comite_distrital_saude.txt
*  atendimento_ao_cidadao.txt
*  regulamentacao.txt

Fontes:

https://www.tjdft.jus.br/ouvidoria

https://www.tjdft.jus.br/informacoes/cidadania/justica-comunitaria/publicacoes/arquivos/Cartilha_JusCom.pdf



# Retrieval Augmented Generation (RAG)


"O futuro da IA reside na engenharia cuidadosa de sistemas. De acordo com Zaharia et al., resultados da Databricks revelam que '60% das aplicações de LLM utilizam alguma forma de RAG, enquanto 30% utilizam cadeias multi-etapas.'"
https://qdrant.tech/articles/rag-is-dead/


Retrieval Augmented Generation (RAG) combina dois tipos de memória: um que é como o conhecimento prévio do modelo e outro que é como um mecanismo de busca, tornando-o mais inteligente no acesso e uso da informação.

O artigo que a primira vez falou de RAG, foi um escrito pelos pesquisadores do Facebook **Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks**: https://arxiv.org/pdf/2005.11401


https://cookbook.openai.com/examples/vector_databases/qdrant/qa_with_langchain_qdrant_and_openai

https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/

#### Vantagens

*   Informações atualizadas
*   Maior confiança na respostas

In [1]:
from IPython.display import Image

In [2]:
Image(url="../images/rag.png", width=800, height=200)

In [3]:
Image(url="../images/fig31.png", width=800, height=200)

### Infraestrutura


Criar uma aplicação de Retrieval Augmented Generation (RAG) exige a integração eficiente de vários componentes complexos. Servidores são elementos da infraestrutura RAG que hospedagem e o gerenciamento de modelos LLMs e modelos de embeddings requerem recursos computacionais significativos. A implementação de RAG envolve a integração de diversos componentes, como recuperadores, geradores e fontes de dados externas. Esses servidores ainda oferecem APIs e endpoints para facilitar a integração das diferentes partes do pipeline RAG, orquestrando o fluxo de trabalho e gerenciando o fluxo de dados entre as etapas de recuperação e geração. Além disso, eles garantem a alocação otimizada de recursos, como GPUs, memória e armazenamento, e lidam com o balanceamento de carga para distribuir a carga de trabalho computacional.

Para rodar local tempos algumas opções de frameworks de inferencia:

*  Ollama

*  llama.cpp

*  GPT4All

*  LM Studio

*  jan.ai

*  h2oGPT

*  localllm


Por questões de praticidade, neste projeto, não vamos rodar o LLM local, utilizaremos o serviço da Nvidia para armazenar o modelo.

# Crie uma conta na Nvidia

### Passo 1:

Vá ao site da Nvidia https://build.nvidia.com/explore/discover e crie uma conta

In [4]:
Image(url="../images/fig32.png", width=800, height=200)

### Passo 2:

Escolha o modelo que quer usar. No curso usaremos o **meta/llama-3.1-405b-instruct**

In [4]:
Image(url="../images/fig34.png", width=800, height=200)

### Passo 3:

**Gere API Key** 

In [5]:
Image(url="../images/fig33.png", width=800, height=200)

Caso esteja executando no Google Colab configure a  **API Key**

Para poder usar a API Key no Google Colab:

*  Abra o Google Colab e vá para Secrets.
*  Digite o Nome e o Valor do segredo. Embora o Valor possa ser alterado, o Nome não pode ser alterado. Neste código sugiro NVIDIA_KEY, mas você pode alterar
*  Ative o acesso ao Notebook.
*  Finalmente, para usá-lo no notebook, use o código fornecido com o nome do seu segredo no lugar de <secretName>

In [7]:
!pip install tiktoken openai langchain-community langchain_text_splitters langchain_huggingface langchain_qdrant qdrant-client

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 1.0 MB/s eta 0:00:00
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     ---------------------------------------- 43.7/43.7 kB 2.1 MB/s eta 0:00:00
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached scikit_learn-1.5.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached scipy-1.14.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/799.3 kB ? eta -:--:--
   ---- -------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.111.1 requires email_validator>=2.0.0, which is not installed.
fastapi 0.111.1 requires python-multipart>=0.0.7, which is not installed.
fastapi 0.111.1 requires uvicorn[standard]>=0.12.0, which is not installed.


In [9]:
import sys
from os import listdir
from os.path import isfile, join, isdir

from langchain_text_splitters import TokenTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import TokenTextSplitter
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

from langchain_community.document_loaders import TextLoader

from langchain_huggingface import HuggingFaceEmbeddings

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain_qdrant import Qdrant

from openai import OpenAI
import tiktoken
#from google.colab import userdata

In [10]:
Image(url="../images/flow-diagram.png", width=600, height=200)

# Fase de Montagem da Estrutura dos Dados Externos

## Etapa 1: Criação de dados externos

Os novos dados, que não fazem parte do conjunto de dados de treinamento original do LLM, são chamados de dados externos. Esses dados podem ser obtidos de múltiplas fontes, como APIs, bancos de dados ou repositórios de documentos. Os dados podem existir em vários formatos, como arquivos, registros de banco de dados ou texto longo.

Fonte: https://aws.amazon.com/pt/what-is/retrieval-augmented-generation/


In [11]:
path = '../data/'
file_list = []
for f in listdir(path):
    file_list.append(join(path, f))
file_list

['../data/atendimento_ao_cidadao.txt',
 '../data/canais_comunicacao.txt',
 '../data/carta_servicos.txt',
 '../data/cartilha_justica_comunitaria.txt',
 '../data/comite_distrital_saude.txt',
 '../data/historico.txt',
 '../data/ouvidoria_tjdft.txt',
 '../data/regulamentacao.txt',
 '../data/sobre_nos.txt']

In [16]:
files = []
for file_path in file_list:
    raw_documents = TextLoader(file_path, encoding='utf-8').load()
    files.append({"content": raw_documents[0].page_content, "path":file_path})
files[5]

{'content': 'Histórico\n\nNo TJDFT, a idéia de Ouvidoria nasceu no ano de 1997 quando a atividade foi estabelecida pela Estrutura Organizacional da Secretaria do Tribunal de Justiça do Distrito Federal e dos Territórios (REORG), aprovada pela Resolução nº 04/97, como sendo de Comunicação Social a ser desenvolvida pela Assessoria de Comunicação Social conforme descrito no art. 7º daquele documento:\n\n"À Assessoria de Comunicação Social - ACS, subordinada à Presidência, compete assessorar o Presidente nas atividades de relações públicas, de ouvidoria , de assessoramento parlamentar e de imprensa; agendar compromissos sociais; promover eventos; manter arquivo jornalístico; zelar pela imagem do TJDFT; cumprir legislação específica e normas regulamentadoras e apresentar relatório anual de atividades desenvolvidas no exercício anterior."\n\nPara atender à determinação do Reorg, em outubro de 1998 a Ouvidoria foi implantada na ACS apenas com o canal Alô-Justiça e Ouvidor Virtual e, em julho 

In [15]:
#file_path = '../data/atendimento_ao_cidadao.txt'
#raw_documents = TextLoader(file_path, encoding='utf-8').load()
#file_content = raw_documents[0].page_content

## Etapa 2: Divisão dos dados em chunks

Existem diversos motivos parea dividir o testo em chunks: dividir o texto em chunks com sobreposição (chunk_overlap) **ajuda a preservar o contexto**; modelos de linguagem têm **limitações no número de tokens** (palavras ou subpalavras) que podem processar de uma vez; processar partes menores de texto pode ser mais **rápido e eficiente**.

Decidir o **tamanho do chunk (chunk_size) e a sobreposição do chunk (chunk_overlap)** é muito importante para evitar a **perda de contexto**. A sobreposição do chunk permite que parte do contexto do chunk anterior seja incluída no próximo chunk, o que possibilita um melhor fluxo e consistência do contexto

 Splitting é o processo de simplesmente dividir seu texto em chunks de tamanho diverso, independentemente do seu conteúdo ou forma.


*   **TokenTextSplitter**: Divide o texto com base no número de tokens, onde um token pode ser uma palavra, pontuação ou subpalavra, dependendo do modelo de tokenização. Útil quando você deseja garantir que cada chunk tenha um número específico de tokens.

*   **CharacterTextSplitter**: Divide o texto com base no número de caracteres. Útil quando você deseja uma divisão mais simples baseada em caracteres, especialmente em cenários onde o comprimento em caracteres é mais relevante do que o número de tokens.

*   **RecursiveCharacterTextSplitter**: Divide o texto (baseado no chunk_size) recursivamente com base em vários separadores, começando pelo primeiro separador na lista e progredindo para o próximo se necessário. Útil para garantir uma divisão mais natural do texto, preservando a integridade das frases e parágrafos. Por exemplo, tenta dividir por parágrafos primeiro, depois por frases, e finalmente por espaços, se necessário.

*   **SentenceTransformersTokenTextSplitter**: Divide o texto usando o tokenizador dos modelos SentenceTransformers. Esses modelos são usados principalmente para gerar embeddings de sentenças. Útil quando se trabalha especificamente com modelos SentenceTransformers e se deseja que a divisão do texto esteja alinhada com a tokenização utilizada por esses modelos.

Playgroud: www.chunkviz.com

https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/a4570f3c4883eb9b835b0ee18990e62298f518ef/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb

Experimente testar com chunk_size e chunk_overlap diferentes, bem como com tipos diferentes de Splitter.

*   chunk_size = 100, chunk_overlap = 50
*   chunk_size = 1000, chunk_overlap = 300

In [17]:
Image(url="../images/chunk.png", width=600, height=200)

In [18]:
chunk_size = 500
chunk_overlap = 50

text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap, separators=["\n", ".", " "], add_start_index = True)
#text_splitter = TokenTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)
#text_splitter = CharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap,separator=' ')
#text_splitter = SentenceTransformersTokenTextSplitter()

chunk_list = []
for file in files:
    chunk_file = text_splitter.split_text(file["content"])
    chunk_list.append({"chunks": chunk_file, "metadata":file["path"]})
chunk_list[6]

{'chunks': ['Ouvidoria do TJDFT\núltima modificação: 01/07/2024 12:18\nA Ouvidoria é um canal de comunicação entre o cidadão e o Tribunal de Justiça do Distrito Federal e dos Territórios - TJDFT. Foi criada para esclarecer dúvidas sobre o funcionamento da instituição, além ser o canal para envio de reclamações, denúncias, sugestões, pedidos de acesso à informação e elogios relativos à atuação do órgão.',
  'O principal objetivo da Ouvidoria é melhorar os nossos serviços, levando em consideração a opinião da população assistida pelo Tribunal.\n\nPor isso, não deixe de entrar em contato para registrar a sua manifestação, pois ela é muito importante para aprimorarmos nossas atividades.\n\nVocê pode falar com a nossa Ouvidoria:\nFormulário eletrônico: tjdft.jus.br/ouvidoria/formulario-eletronico.\nPor telefone: 0800 6146466, das 12h às 18h30, em dias úteis.\nPor e-mail: ouvidoria@tjdft.jus.br.',
  'Por e-mail: ouvidoria@tjdft.jus.br.\nPor correspondência: TJDFT - Ouvidoria-Geral - Praça Mu

In [19]:
#text_splitter = TokenTextSplitter(chunk_size = 500, chunk_overlap = 50)
#chunks = text_splitter.split_text(file_content)
#metadata = [{"path": file_path} for _ in chunks]

## Etapa 3: Criação da banco de dados vetorial e armazenamento dos embeddings

### Definião do modelo de embeddings

Uma embedding é uma representação vetorial densa de dados. Embeddings transformam dados de alta dimensionalidade, como palavras, frases, ou até mesmo imagens, em vetores de menor dimensionalidade que capturam as características semânticas dos dados originais.


https://github.com/infoslack/qdrant-example/blob/main/Embeddings.ipynb

In [20]:
Image(url="../images/word2.jpg", width=600, height=200)

In [21]:
Image(url="../images/word.png", width=600, height=200)

In [22]:
# Carga e configuração do modelo de embeddings
embeddings = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/msmarco-bert-base-dot-v5",
    model_kwargs = {'device': 'cpu'},
    encode_kwargs = {'normalize_embeddings': True})  # indica que os embeddings devem ser normalizados (ajustados para que tenham magnitude 1)

C:\Users\joelma\anaconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


### Criação do banco

In [23]:
db_path = "../vectodb"
db_name = "VectorDB"

clientq = QdrantClient(path=db_path)

if clientq.collection_exists(db_name):
    clientq.delete_collection(db_name)

clientq.create_collection(
    db_name,
    vectors_config=VectorParams(size=768, distance=Distance.DOT)
    #vectors_config=VectorParams(size=768, distance=Distance.EUCLID)
    #vectors_config=VectorParams(size=768, distance=Distance.COSINE)
)

True

**VectorParams**

*   size=768: Define o tamanho dos vetores que serão armazenados na coleção. Neste caso, os vetores terão 768 dimensões. Este tamanho deve corresponder ao tamanho dos embeddings gerados pelo modelo de linguagem que você está utilizando.

*   distance=Distance.DOT: Define a métrica de distância que será usada para calcular a similaridade entre os vetores. Distance.DOT especifica que a distância de produto escalar (dot product) será usada.


Teste com utras opções de distãncia de similaridade:
*  Distance.EUCLID (distância euclidiana)
*  Distance.COSINE (distância cosseno)

In [24]:
Image(url="../images/cos.png", width=600, height=200)

### Geração e armazenamento dos embeddings


In [25]:
qdrant = Qdrant(clientq, db_name, embeddings)

C:\Users\joelma\anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.1.2 and will be removed in 0.5.0. Use QdrantVectorStore instead.
  warn_deprecated(


In [26]:
# Separação dos chunks de cada arquivo
for chunks in chunk_list:
    file_path = chunks['metadata']
    chunks = chunks['chunks']
    metadata = [{"path": file_path} for _ in chunks]

    #Função built-in que gera as embeddings para o texto e as carrega no banco vetoriaç
    qdrant.add_texts(chunks, metadatas = metadata)


In [27]:
Image(url="../images/vcdb.png", width=600, height=200)

In [28]:
# Teste em apenas um arquivo
#qdrant = Qdrant(clientq, db_name, llm_embedding)
#qdrant.add_texts(chunks, metadatas = metadata)

# Fase de Utilização do RAG

### Etapa 1: Definição do modelo de embedding das perguntas

In [29]:
embeddings = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/msmarco-bert-base-dot-v5",
    model_kwargs = {'device': 'cpu'},
    encode_kwargs = {'normalize_embeddings': True})

### Etapa 2: Conexão com o banco de dados vetorial

In [30]:
#db_path = "./vectodb"
#db_name = "VectorDB"
#clientq = QdrantClient(path=db_path)
qdrant = Qdrant(clientq, db_name, embeddings)

### Etapa 3: Conexão com o servidor

In [31]:
client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  #api_key = userdata.get('NVIDIA_KEY')   # Se estiver executando no COLAB
  #api_key = "COLOQUE AQUI SUA KEY"       # Se estiver executando LOCAL
  api_key = "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"       # Se estiver executando LOCAL
)

### Etapa 4: Leitura da pergunta do usuário e busca dos embeddings mais similares (Retriever)

In [32]:
#./data/ouvidoria_tjdft.txt
#Resposta esperada:  Foi criada para esclarecer dúvidas sobre o funcionamento da instituição, além ser o canal para envio de reclamações, denúncias, sugestões, pedidos de acesso à informação e elogios relativos à atuação do órgão.
query = "Como a ouvidoria pode me ajudar?"
top_result_context = qdrant.similarity_search(query = query, k = 3)

In [33]:
#./data/atendimento_ao_cidadao.txt
# Resposta esperada: O Tribunal tem até 20 dias para responder ao pedido, prazo que pode ser prorrogado por mais 10 dias, mediante justificativa expressa.
query = "Em qual prazo em média obtenho resposta à minha demanda?"
top_result_context = qdrant.similarity_search(query = query, k = 3)

In [34]:
#./data/regulamentacao.txt
# Resposta esperada: Portaria GPR 3028 de 19/12/2022
query = "Qual a portaria que fala sobre a revisão e a atualização da Transparência e Prestação de Contas?"
top_result_context = qdrant.similarity_search(query = query, k = 3)

In [35]:
#./data/comite_distrital_saude.txt
query = "Para que serve o Comitê Distrital da Saúde?"
top_result_context = qdrant.similarity_search(query = query, k = 3)

In [36]:
#./data/canais_comunicacao.txt
# Resposta esperada: judicializacao.sesdf@gmail.com
query = "Qual o email do Núcleo de Judicialização?"
top_result_context = qdrant.similarity_search(query = query, k = 3)

In [37]:
#./data/sobre_nos.txt
# Resposta esperada: A estrutura organizacional da unidade está ilustrada na imagem seguinte e pode ser consultada no site do Tribunal.
query = "Qual a estrutura organizacional?"
top_result_context = qdrant.similarity_search(query = query, k = 3)

### Etapa 5: Construção do contexto com os embeddings mais similares

A idéia é guiar o LLM para montar a resposta com base no contexto dos embeddings similares a pergunta.

In [38]:
response_list = []
context = ""
mappings = {}

for i, res in enumerate(top_result_context):
    context += f"{i}\n{res.page_content}\n\n"
    mappings[i] = res.metadata.get("path")
    response_list.append({"id": i, "path": res.metadata.get("path"), "content": res.page_content})

In [39]:
context

'0\nEstrutura organizacional da Ouvidoria:\nA estrutura organizacional da unidade está ilustrada na imagem seguinte e pode ser consultada no site do Tribunal.\n\norganograma.png\n\nCOMPOSIÇÃO\nOuvidora-Geral do Tribunal de Justiça do DF e dos Territórios\nDesembargadora Maria de Lourdes Abreu\n\nOuvidor-Geral Substituto do Tribunal de Justiça do DF e dos Territórios\nDesembargador José Firmo Reis Soub\n\nCoordenação\nMarília Barbosa de Barcelos - Secretária da Ouvidoria-Geral\n\n1\nQual o prazo para envio da resposta à minha demanda?\n\n2\n.\n\n'

### Etapa 6: Definição do prompt e mensagem a ser enviado ao LLM

In [40]:
prompt = {"role": "system",
          "content": "Responda à pergunta do usuário usando documentos fornecidos no contexto. No contexto estão documentos que devem conter uma resposta. Use apenas o contexo para responder. Monte uma resposta curta e objetiva. Responda sempre em português."}
messages = [prompt, {"role": "user", "content": f"Documents:\n{context}\n\nQuestion: {query}"}]


### Etapa 7: Acesso ao LLM escolhido

Parâmetros:

*  **model:** Nome do modelo a ser utilizado.
*  **messages:** Lista de mensagens que formam o contexto da conversa.
*  **temperature:** Controla a aleatoriedade da geração de texto.  Valores mais baixos tornam a resposta mais determinística, enquanto valores mais altos aumentam a diversidade.
*  **top_p:** Controla a amostragem de núcleo para limitar a probabilidade acumulada. Define o limite de probabilidade acumulada para considerar as próximas palavras. Se top_p=1, considera todas as palavras possíveis.
*  **max_tokens:** Define o número máximo de tokens (palavras ou subpalavras) que podem ser gerados na resposta.
*  **stream:** Define se a resposta será transmitida em tempo real ou retornada de uma vez.

In [41]:
answer = client.chat.completions.create(
  model="meta/llama-3.1-405b-instruct",
  messages=messages,
  temperature=0.2,
  top_p=1,
  max_tokens=1024,
  stream=False
)

In [42]:
print(f'"Resposta": {answer.choices[0].message.content}')

"Resposta": A estrutura organizacional da Ouvidoria está ilustrada no organograma e pode ser consultada no site do Tribunal.


In [43]:
print(f'"Contexto": {response_list[0]}')
print(f'"Contexto": {response_list[1]}')
print(f'"Contexto": {response_list[2]}')

"Contexto": {'id': 0, 'path': '../data/sobre_nos.txt', 'content': 'Estrutura organizacional da Ouvidoria:\nA estrutura organizacional da unidade está ilustrada na imagem seguinte e pode ser consultada no site do Tribunal.\n\norganograma.png\n\nCOMPOSIÇÃO\nOuvidora-Geral do Tribunal de Justiça do DF e dos Territórios\nDesembargadora Maria de Lourdes Abreu\n\nOuvidor-Geral Substituto do Tribunal de Justiça do DF e dos Territórios\nDesembargador José Firmo Reis Soub\n\nCoordenação\nMarília Barbosa de Barcelos - Secretária da Ouvidoria-Geral'}
"Contexto": {'id': 1, 'path': '../data/atendimento_ao_cidadao.txt', 'content': 'Qual o prazo para envio da resposta à minha demanda?'}
"Contexto": {'id': 2, 'path': '../data/historico.txt', 'content': '.'}


In [ ]:
!streamlit run ../app_model.py

# Para uma resposta em tempo real (streaming)

In [44]:
def handle_streaming_response(stream_response):
    answer = ""
    for chunk in stream_response:
        if chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            answer += content
            print(content, end='')
    return answer

stream_response = client.chat.completions.create(
    model="meta/llama-3.1-405b-instruct",
    messages=messages,
    temperature=0.2,
    top_p=1,
    max_tokens=1024,
    stream=True
)

answer = handle_streaming_response(stream_response)
print(f'"Resposta": {answer}')


A estrutura organizacional da Ouvidoria está ilustrada no organograma e pode ser consultada no site do Tribunal."Resposta": A estrutura organizacional da Ouvidoria está ilustrada no organograma e pode ser consultada no site do Tribunal.


# Métricas de avaliação


1. **Fidelidade**: grau em que o texto gerado reflete com precisão as informações presentes nos documentos de origem recuperados.

2. **Relevância das Respostas**: Mede a relevância das respostas geradas para as questões colocadas. Determina a utilidade do pipeline RAG em aplicações práticas.

3.  **Recuperação de Contexto**: Avalia a capacidade do sistema RAG de recuperar todas as informações relevantes das fontes de dados externas. Uma elevada recuperação de contexto é indicativa de um sistema que pode utilizar de forma abrangente os dados disponíveis.

4.  **Precisão do Contexto**: Em contraste com a recuperação, a precisão do contexto mede a proporção de informação recuperada que é relevante para a tarefa em questão. Essa métrica garante que o sistema RAG filtre com eficiência dados estranhos, concentrando-se na qualidade e não na quantidade em seu processo de aumento.

5.  **Relevância do Contexto**: Combina aspectos de recuperação e precisão, avaliando a relevância geral do contexto utilizado pelo sistema RAG. Esta métrica sublinha a importância de uma abordagem equilibrada à recuperação de dados, onde tanto a amplitude como a especificidade da informação são otimizadas.

6.  **Similaridade Semântica das Respostas**: Esta métrica avalia o alinhamento semântico entre as respostas geradas e a verdade básica (ou respostas esperadas), levando em consideração as nuances da linguagem. É essencial verificar se o sistema RAG capta os significados e não apenas os aspectos superficiais das respostas.

7.  **Correção das Respostas**: Além da relevância e semelhança semântica, a correção das respostas avalia diretamente a precisão das informações fornecidas no texto gerado. Esta métrica é fundamental para garantir que os LLMs aumentados por RAG atuem como fontes confiáveis de informação.

https://blog.dsacademy.com.br/principais-metricas-para-avaliar-pipelines-rag-com-llms/